In [ ]:
from airflow.decorators import dag, task
from datetime import datetime, timedelta
import sys
import pandas as pd
sys.path.append('/opt/airflow')
from src.pipeline_tempo import extrair_dados, processar_dados

@dag(
    dag_id = "pipeline_clima",
    schedule = "*/1 * * * *",
    start_date = datetime.now(),
    catchup = False, #TODO: ve se compenssa pq da pra tentar pegar os passados(porém perde a funcionalidade principal)
    tags = ['clima', 'previsao_tempo', 'caxias do sul'],
    default_args={
        'owner': 'guiandreis',
        'retries': 1,
        'retry_delay': timedelta(minutes=5)
    })#revisar esse      

def pipeline_tempo():
    """DAG para extrair dados climáticos da API Visual Crossing, 
    processá-los e salva-los"""
    
    @task()
    def extrair():
        print("📥 Extraindo dados da API...")
        df_raw = extrair_dados('Caxias do Sul', dias=7)
        
        #salvando raw
        path_raw = '/opt/airflow/data/raw/previsao_tempo.csv'
        df_raw.to_csv(path_raw, index=False)
        print(f"✅ Dados brutos salvos em {path_raw}")
        return path_raw
    
    @task()
    def processar(path_raw):
        """Processando e limpando os dados e salvando"""
        print("⚙️ Processando dados...")
        df_raw = pd.read_csv(path_raw)
        df_limpo = processar_dados(df_raw)
        
        #salvando processado
        path_processed = '/opt/airflow/data/processed/previsao_tempo_limpo.csv'
        df_limpo.to_csv(path_processed, index=False)
        print(f"✅ Dados processados salvos em {path_processed}")
        return path_processed
    
    #fluxo de execução
    path_raw = extrair()
    path_processed = processar(path_raw)

pipeline_tempo_dag = pipeline_tempo()
